In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn import preprocessing
from sklearn import metrics
import pandas_datareader.data as web

In [2]:
#Read stock data use pandas_datareader.data from web
# Get Stock Data
stk_tickers = ['MSFT', 'IBM', 'GOOGL']
ccy_tickers = ['DEXJPUS', 'DEXUSUK']
idx_tickers = ['SP500', 'DJIA', 'VIXCLS']
stk_data = web.DataReader(stk_tickers, 'yahoo')
ccy_data = web.DataReader(ccy_tickers, 'fred')
idx_data = web.DataReader(idx_tickers, 'fred')

In [5]:
# Select columns

Base = stk_data.loc[:, ('Adj Close', 'MSFT')]
X1 = stk_data.loc[:, ('Adj Close', ('GOOGL', 'IBM'))]
X2 = ccy_data
X3 = idx_data
X1

Attributes   Adj Close            
Symbols          GOOGL         IBM
Date                              
2017-09-15   46.764500  109.150810
2017-09-18   46.487499  108.947289
2017-09-19   46.842999  108.826691
2017-09-20   47.376999  109.949684
2017-09-21   47.377499  109.482384
...                ...         ...
2022-09-09  110.650002  129.190002
2022-09-12  110.860001  130.660004
2022-09-13  104.320000  127.250000
2022-09-14  105.000000  127.690002
2022-09-15  103.790001  125.699997

[1259 rows x 2 columns]